In [2]:
import gym

In [3]:
env = gym.make("CartPole-v0")

In [4]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [5]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [6]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [7]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 84
Penalties incurred: 22


In [8]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'].getvalue())
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

AttributeError: 'str' object has no attribute 'getvalue'

In [9]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [10]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

Wall time: 33.6 s


In [11]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 13.03
Average penalties per episode: 0.0


In [14]:
env.action_space.sample()

1

In [18]:
np.shape(q_table)

(500, 6)

In [8]:
env.observation_space.shape[0]

4

In [10]:
import numpy as np

In [46]:
memory = np.zeros((200,env.observation_space.shape[0]*2+env.action_space.n))

In [47]:
memory.shape

(200, 10)

In [48]:
memory_counter = 0
cur_state = env.reset()
cur_state

array([0.00206155, 0.01060546, 0.04419723, 0.04416976])

In [57]:
def step_on(env,cur_state,memory_counter):
    action = env.action_space.sample()
    new_state, reward, done, _ = env.step(action)
    memory[memory_counter] = np.hstack([cur_state, action, reward, new_state])
    if done:
        cur_state = env.reset()
    else:
        cur_state = new_state

In [59]:
for i in range(0,200):
    step_on(env,cur_state,i)

In [62]:
memory[2]

array([ 0.00206155,  0.01060546,  0.04419723,  0.04416976,  0.        ,
        1.        , -0.01411324, -0.15437894, -0.01183131,  0.32408894])

In [69]:
env.action_space.sample()

0

In [106]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input

In [107]:
model = tf.keras.models.Sequential()

In [111]:
model.add(Input(shape = (1,4)))
model.add(Dense(64,activation = 'relu'))
model.add(Dense(32,activation = 'relu'))
model.add(Dense(env.action_space.n,activation = 'relu'))

In [112]:
cur_state

array([0.00206155, 0.01060546, 0.04419723, 0.04416976])

In [113]:
model.predict(cur_state)

ValueError: Error when checking input: expected input_1 to have shape (4,) but got array with shape (1,)